In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [3]:
train_dir = r'C:/Users/DELL/Downloads/zdataset/Alzheimer_s Dataset'  
test_dir = r'C:/Users/DELL/Downloads/zdataset/Alzheimer_s Dataset'

In [4]:
input_shape = (128, 128, 3)
batch_size = 32

In [5]:
data_generator = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 6400 images belonging to 2 classes.


In [7]:
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 6400 images belonging to 2 classes.


In [8]:
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  
train_labels = train_generator.classes

  3/200 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step  

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step


In [10]:
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step


In [11]:
ada_classifier = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1), 
                                    n_estimators=50,
                                    learning_rate=1.0,
                                    algorithm='SAMME.R')
ada_classifier.fit(train_features, train_labels)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1))

In [12]:
test_predictions = ada_classifier.predict(test_features)

In [13]:
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 97.83%


In [14]:
model.save('cnn_feature_extractor7.h5')

In [15]:
joblib.dump(ada_classifier, 'ada_classifier.pkl')

['ada_classifier.pkl']

In [16]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image

cnn_model = load_model('cnn_feature_extractor7.h5')
dt_classifier = joblib.load('ada_classifier.pkl')

def classify_image(img_path):
    img = image.load_img(img_path, target_size=(128, 128))  
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  
    img_array /= 255.0  

    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  

    prediction = dt_classifier.predict(features)
    
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented',2: 'NonDemented',3:'VeryMildDemented'} 
    result = class_labels[prediction[0]]
    return result

img_path = r'C:/Users/DELL/Downloads/zdataset/Alzheimer_s Dataset/test/MildDemented/30 (4).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
The image is classified as: MildDemented
